# Advanced pandas

In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

## 12.1 Categorical Data

### Background and Motivation

In [2]:
values = pd.Series(['apple','orange','apple',
                   'apple'] * 2)

In [3]:
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [4]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [5]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [6]:
#许多数据系统都发展出了特定的表示重复值的方法，以进行高效的存储和计算
#在数据仓库中，最好的方式是使用包含不同值的维表(dimension table),将主要的
#参数存储为引用维表整数键
values = pd.Series([0,1,0,0] * 2)
dim = pd.Series(['apple','orange'])
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [7]:
dim

0     apple
1    orange
dtype: object

In [8]:
#使用take方法存储原始的字符串series
#这种用整数表示的方法称为分类，表示分类的整数值称为分类编码
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### Categorical Type in pandas

In [25]:
#pandas有一个特殊的分类类型，用于保存使用整数分类表示法的数据
fruits = ['apple','orange','apple','apple','apple'] * 2
N = len(fruits)

In [26]:
df = pd.DataFrame({'fruit':fruits,
                  'basket_id':np.arange(N),
                  'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,12,1.594976
1,1,orange,14,3.229155
2,2,apple,10,2.508377
3,3,apple,9,3.631700
4,4,apple,3,2.225589
5,5,apple,5,3.359677
6,6,orange,12,0.201952
7,7,apple,14,3.224939
8,8,apple,13,3.723263
9,9,apple,4,1.456118


In [27]:
a = df['fruit']
type(a.values)

numpy.ndarray

In [28]:
#df['fruit']是一个python字符串对象的数组
#调用astype('category')可以将它的dtype转换为分类
fruit_cat = df['fruit'].astype('category')

In [29]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5     apple
6    orange
7     apple
8     apple
9     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [30]:
#fruit_cat的值不是一个numpy数组，而是一个pandas.Categorical实例
c = fruit_cat.values

In [31]:
type(c)

pandas.core.arrays.categorical.Categorical

In [32]:
#分类对象有categories和codes属性
c.categories

Index(['apple', 'orange'], dtype='object')

In [33]:
c.codes

array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0], dtype=int8)

In [34]:
#将得到的分类对象赋值给df的fruit列
df['fruit'] = df['fruit'].astype('category')

In [35]:
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5     apple
6    orange
7     apple
8     apple
9     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [36]:
#直接创建pandas.Categorical
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [39]:
my_categories.codes  #自动创建codes

array([2, 0, 1, 2, 0], dtype=int8)

In [37]:
#已知分类编码,创建pandas.Categorical
categories = ['foo','bar','baz']
codes = [0,1,2,0,0,1]
my_cats_2 = pd.Categorical.from_codes(codes,categories)

In [38]:
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [41]:
#指定分类一个有意义的顺序
ordered_cat = pd.Categorical.from_codes(codes,categories,ordered=True)
ordered_cat
#结果表明，foo在bar前面，bar在baz前面

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [42]:
#无序的分类实例可以调用as_ordered方法排序
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

### Computations with Categoricals

In [43]:
np.random.seed(12345)

In [44]:
draws = np.random.randn(1000)
draws[:5]

array([-0.2047,  0.4789, -0.5194, -0.5557,  1.9658])

In [45]:
#计算数据的分位面元，提取一些统计信息
bins = pd.qcut(draws,4)

In [46]:
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [49]:
#为面元设置label
bins = pd.qcut(draws,4,labels=['Q1','Q2','Q3','Q4'])

In [50]:
bins
#可以看到bins是categories对象

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [52]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [53]:
bins = pd.Series(bins,name='quartile')

In [54]:
bins

0      Q2
1      Q3
2      Q2
3      Q2
4      Q4
5      Q4
6      Q3
7      Q3
8      Q4
9      Q4
       ..
990    Q1
991    Q4
992    Q1
993    Q4
994    Q2
995    Q3
996    Q2
997    Q1
998    Q3
999    Q4
Name: quartile, Length: 1000, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [55]:
results = (pd.Series(draws).groupby(bins).agg(['count','min','max']))

In [63]:
results = results.reset_index()

In [64]:
results['quartile']  #分位数列保留了原始的面元分类信息

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

### Better performance with categoricals

In [65]:
#在一个特定数据集上做大量的分析，将其转换为分类可以极大地提高效率
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [68]:
categories = labels.astype('category')

In [69]:
#标签使用的内存比分类多得多
labels.memory_usage()

80000080

In [70]:
categories.memory_usage()

10000272

In [71]:
#转换为分类不是没有代价的，但这是一次性的代价
%time _ = labels.astype('category')

CPU times: user 273 ms, sys: 14.9 ms, total: 288 ms
Wall time: 287 ms


### Categorical Methods

In [72]:
s = pd.Series(['a','b','c','d'] * 2)
cat_s = s.astype('category')

In [75]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [78]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [79]:
#包含分类数据的series的cat属性提供了分类方法的入口
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [81]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [82]:
#假设我们知道这个数据的实际分类超过过数据中的四个值，可以使用set_categories方法改变它们
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)

In [83]:
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [85]:
#在这个例子中虽然结果没什么变化
cat_s2.cat.categories

Index(['a', 'b', 'c', 'd', 'e'], dtype='object')

In [86]:
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [87]:
cat_s2.value_counts()  #多了e的分类

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [88]:
#在过滤的series数据中，可能一些分类已经没有了，可以删除这些分类
cat_s3 = cat_s[cat_s.isin(['a','b'])]

In [89]:
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [90]:
cat_s.isin(['a','b'])

0     True
1     True
2    False
3    False
4     True
5     True
6    False
7    False
dtype: bool

In [91]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

![categories](jietu/categories.png)

### Creating dummy variables for modeling

In [92]:
#通常会将分类数据转换为虚拟变量，也称为one-hot编码
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

In [94]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [95]:
#get_dummies()可以转换分类数据的series为含有虚拟变量的dataframe
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## 11.2 Advanced GroupBy Use

### Group Transforms and "Unwrapped" GroupBys

In [124]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [125]:
gr = df.groupby('key')

In [126]:
for key,group in gr:
    print(key)
    print(group)

a
  key  value
0   a    0.0
3   a    3.0
6   a    6.0
9   a    9.0
b
   key  value
1    b    1.0
4    b    4.0
7    b    7.0
10   b   10.0
c
   key  value
2    c    2.0
5    c    5.0
8    c    8.0
11   c   11.0


In [127]:
g = df.groupby('key').value

In [129]:
for key,group in g:
    print(key)
    print(group)

a
0    0.0
3    3.0
6    6.0
9    9.0
Name: value, dtype: float64
b
1      1.0
4      4.0
7      7.0
10    10.0
Name: value, dtype: float64
c
2      2.0
5      5.0
8      8.0
11    11.0
Name: value, dtype: float64


In [130]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [101]:
#产生一个和df['value']形状相同的series,但值替换为按键分组的平均值
#transform返回结果和输入组形状相同
g.transform(lambda x:x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [104]:
g.transform('mean')   #与上面结果一样

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [132]:
g.transform(lambda x:x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [133]:
#计算每个分组的降序排名
g.transform(lambda x:x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [134]:
def normalize(x):
    return (x-x.mean()) / x.std()

In [137]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [138]:
g.apply(normalize) #与上面结果一样

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [117]:
arr = np.array([0,3,6,9])

In [143]:
arr1 = np.array([1,4,7,10])
arr2 = np.array([2,5,8,11])

In [136]:
normalize(arr)

array([-1.3416, -0.4472,  0.4472,  1.3416])

In [142]:
normalize(arr1)

array([-1.3416, -0.4472,  0.4472,  1.3416])

In [144]:
normalize(arr2)

array([-1.3416, -0.4472,  0.4472,  1.3416])

In [147]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [148]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')

In [149]:
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

### Grouped Time Resampling

In [150]:
#resample方法是一个基于内在时间的分组操作
N = 15
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time': times,
                   'value': np.arange(N)})
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [152]:
tsr = df.set_index('time').resample('5min')

In [155]:
for key,group in tsr:
    print(key)
    print(group)
    print('****')

2017-05-20 00:00:00
                     value
time                      
2017-05-20 00:00:00      0
2017-05-20 00:01:00      1
2017-05-20 00:02:00      2
2017-05-20 00:03:00      3
2017-05-20 00:04:00      4
****
2017-05-20 00:05:00
                     value
time                      
2017-05-20 00:05:00      5
2017-05-20 00:06:00      6
2017-05-20 00:07:00      7
2017-05-20 00:08:00      8
2017-05-20 00:09:00      9
****
2017-05-20 00:10:00
                     value
time                      
2017-05-20 00:10:00     10
2017-05-20 00:11:00     11
2017-05-20 00:12:00     12
2017-05-20 00:13:00     13
2017-05-20 00:14:00     14
****


In [156]:
tsr.count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [169]:
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3.)})
df2[:20]

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0
7,2017-05-20 00:02:00,b,7.0
8,2017-05-20 00:02:00,c,8.0
9,2017-05-20 00:03:00,a,9.0


In [160]:
#Numpy的 tile() 函数，就是将原矩阵横向、纵向地复制。tile 是瓷砖的意思，
#顾名思义，这个函数就是把数组像瓷砖一样铺展开来。
mat = np.array([[1,2], [3, 4]])

In [161]:
mat

array([[1, 2],
       [3, 4]])

In [162]:
np.tile(mat,(1,4))  #横向

array([[1, 2, 1, 2, 1, 2, 1, 2],
       [3, 4, 3, 4, 3, 4, 3, 4]])

In [163]:
np.tile(mat,(4,1))  #纵向

array([[1, 2],
       [3, 4],
       [1, 2],
       [3, 4],
       [1, 2],
       [3, 4],
       [1, 2],
       [3, 4]])

In [166]:
#对每个key值进行相同的重采样，引入pandas.TimeGrouper对象
time_key = pd.Grouper(freq='5min')
#使用Grouper的限制是时间必须是series或dataframe的索引

In [167]:
resampled = (df2.set_index('time').groupby(['key',time_key]).sum())

In [168]:
resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

In [170]:
resampled.reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0


## 12.3 Techniques for Method Chaining

In [ ]:
#链式编程在两边加()

```python
df = load_data()
df2 = df[df['col2'] < 0]
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()
```

```python
# Usual non-functional way
df2 = df.copy()
df2['k'] = v

# Functional assign way
df2 = df.assign(k=v)
```

In [171]:
#df.assign(** kwargs)#创建或修改列并添加到原数据中
s=pd.Series([11,12,13],name='S')
data=np.arange(21,24)
df=pd.DataFrame({'A':[31,32,33],'B':[41,42,43]})
fun=lambda x:x.A+x.B
 
df.assign(C=fun,D=df.A+df.B,E=s,F=data)#增加新列

,A,B,C,D,E,F
0,31,41,72,72,11,21
1,32,42,74,74,12,22
2,33,43,76,76,13,23


In [172]:
df.assign(C=fun,B=data)#修改原数据

,A,B,C
0,31,21,72
1,32,22,74
2,33,23,76


In [173]:
df.assign(B=data,C=fun)#修改原数据 注意顺序不同，结果不同

,A,B,C
0,31,21,52
1,32,22,54
2,33,23,56


```python
result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
          .groupby('key')
          .col1_demeaned.std())
```

```python
df = load_data()
df2 = df[df['col2'] < 0]
```

```python
df = (load_data()
      [lambda x: x['col2'] < 0])
```

```python
result = (load_data()
          [lambda x: x.col2 < 0]
          .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())
          .groupby('key')
          .col1_demeaned.std())
```

### The pipe Method

```python
a = f(df, arg1=v1)
b = g(a, v2, arg3=v3)
c = h(b, arg4=v4)
```

```python
result = (df.pipe(f, arg1=v1)
          .pipe(g, v2, arg3=v3)
          .pipe(h, arg4=v4))
```

```python
g = df.groupby(['key1', 'key2'])
df['col1'] = df['col1'] - g.transform('mean')
```

```python
def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result
```

```python
result = (df[df.col1 < 0]
          .pipe(group_demean, ['key1', 'key2'], ['col1']))
```